# Spotify Data Visualization

## 1.1 Importing data and libraries

In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from dask import dataframe as dd
%matplotlib inline

In [2]:
# reading a file as a pandas dataframe requires 13 GB+ memory alocation. The file loaded this way is too large.
# reading a file as a dask dataframe due to size with a dtypes choosed by default pandas reader
# more about dask in dataframes: https://docs.dask.org/en/stable/dataframe.html

dask_dataframe = dd.read_csv('spotify_charts.csv', 
                     dtype=object)

## 1.2 Data exploration and optimization stage

In [3]:
# dask dataframe size info

dask_dataframe.info(memory_usage='deep')


<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, title to streams
dtypes: object(9)
memory usage: 1.8 GB


In [4]:
dask_dataframe.head()

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956


In [5]:
# checking the number of unique values 

for col in dask_dataframe.columns:
    print(f'unique values in {col}: {dask_dataframe[col].nunique().compute()}')
    
# its longer to compute every single column in dask dataframe but working with a 13 GB pandas dataframe is avoided. 

unique values in title: 164758
unique values in rank: 200
unique values in date: 1826
unique values in artist: 96115
unique values in url: 217644
unique values in region: 70
unique values in chart: 2
unique values in trend: 4
unique values in streams: 788013


In [6]:
# changing of data types from object to category/int for elements that have relatively few unique values (in comparison to number of all entries)

to_category_cols = ['title', 'url', 'artist', 'region', 'trend', 'chart']
to_datetime64_cols = ['date']
to_int32_cols = ['rank']

for col in to_category_cols:
    dask_dataframe[col] = dask_dataframe[col].astype('category')
        
for col in to_int32_cols:
    dask_dataframe[col] = dask_dataframe[col].astype('int32')
    
for col in to_datetime64_cols:
    dask_dataframe[col] = dask_dataframe[col].astype('datetime64')


In [7]:
# rechecking dask dataframe size info

dask_dataframe.info(memory_usage='deep')

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, title to streams
dtypes: category(6), datetime64[ns](1), object(1), int32(1)
memory usage: 813.4 MB


In [8]:
# pandas dataframe size info

dataframe = dask_dataframe.compute()

dataframe.info(memory_usage='deep')

# file is much smaller and after first steps of optimization is possible to work on pandas dataframe again

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26147953 entries, 0 to 498950
Data columns (total 9 columns):
 #   Column   Dtype         
---  ------   -----         
 0   title    category      
 1   rank     int32         
 2   date     datetime64[ns]
 3   artist   category      
 4   url      category      
 5   region   category      
 6   chart    category      
 7   trend    category      
 8   streams  object        
dtypes: category(6), datetime64[ns](1), int32(1), object(1)
memory usage: 2.2 GB


In [9]:
# checking the number of NaN values 

for col in dataframe.columns:
    print(f'NaN values in {col}: {dataframe[col].isna().sum()}')
    

NaN values in title: 11
NaN values in rank: 0
NaN values in date: 0
NaN values in artist: 18
NaN values in url: 0
NaN values in region: 1
NaN values in chart: 1
NaN values in trend: 1
NaN values in streams: 5826049


In [10]:
# preparing a NaN values filters

title_is_nan = dataframe['title'].isna()
artist_is_nan = dataframe['artist'].isna()
region_is_nan = dataframe['region'].isna()
chart_is_nan = dataframe['chart'].isna()
trend_is_nan = dataframe['trend'].isna()
streams_is_nan = dataframe['streams'].isna()


In [11]:
# NaN data exploratiom
dataframe[title_is_nan].head(3)

,title,rank,date,artist,url,region,chart,trend,streams
72604,NaN,120,2019-06-24,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo...,Japan,top200,NEW_ENTRY,11942
169157,NaN,167,2019-06-25,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo...,Japan,top200,MOVE_DOWN,10310
246763,NaN,128,2019-06-26,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo...,Japan,top200,MOVE_UP,11620


In [12]:
# NaN data exploratiom
dataframe[artist_is_nan].head(3)

,title,rank,date,artist,url,region,chart,trend,streams
291605,NO GOOD,10,2020-07-13,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,NEW_ENTRY,NaN
311398,NO GOOD,10,2020-07-14,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION,NaN
335035,NO GOOD,10,2020-07-15,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION,NaN


In [13]:
# NaN data exploratiom
dataframe[region_is_nan].head(3)

,title,rank,date,artist,url,region,chart,trend,streams
498950,Tengo La Personalidad,47,2021-07-24,Chikybombom La Pantera,https://open.spotify.com/trac,NaN,NaN,NaN,NaN


In [14]:
# NaN data exploratiom
dataframe[chart_is_nan].head(3)

,title,rank,date,artist,url,region,chart,trend,streams
498950,Tengo La Personalidad,47,2021-07-24,Chikybombom La Pantera,https://open.spotify.com/trac,NaN,NaN,NaN,NaN


In [15]:
# NaN data exploratiom
dataframe[trend_is_nan].head(3)

,title,rank,date,artist,url,region,chart,trend,streams
498950,Tengo La Personalidad,47,2021-07-24,Chikybombom La Pantera,https://open.spotify.com/trac,NaN,NaN,NaN,NaN


In [16]:
# NaN data exploratiom
dataframe[streams_is_nan].sample(5)

,title,rank,date,artist,url,region,chart,trend,streams
143534,Banana (feat. Shaggy) - DJ FLe - Minisiren Remix,13,2020-06-22,Conkarah,https://open.spotify.com/track/0dZpw3h6KZhcHec...,Global,viral50,MOVE_UP,NaN
252273,Mi Thimonis,17,2020-10-17,Petros Iakovidis,https://open.spotify.com/track/5vAN1MJikSqkXxC...,Greece,viral50,MOVE_DOWN,NaN
391987,Fendi,20,2020-08-13,Rakhim,https://open.spotify.com/track/6Y6J8CNHtOuklPh...,Lithuania,viral50,SAME_POSITION,NaN
262679,Trippin' (feat. Khalid),14,2018-07-22,Buddy,https://open.spotify.com/track/6TrzifGqGOc2DfB...,Canada,viral50,MOVE_DOWN,NaN
71365,Lebih Dari Egoku,28,2019-12-01,Mawar de Jongh,https://open.spotify.com/track/4vh193RZbEEBanS...,India,viral50,MOVE_UP,NaN


In [17]:
# checking if 'streams' contains any '0' values

streams_is_null = dataframe['streams'] == 0

len(dataframe[streams_is_null])



0

In [18]:
# replacing NaN values with a zeros without losing any data

dataframe['streams'].fillna(value=0, inplace=True)

In [19]:
# changing of data types of 'streams' from float64 to int32

dataframe['streams'] = dataframe['streams'].astype('int32')

In [20]:
# droping rest records containing NaN values
dataframe.dropna(inplace=True)


In [21]:
# rechecking the number of NaN values 

for col in dataframe.columns:
    print(f'NaN values in {col}: {dataframe[col].isna().sum()}')

NaN values in title: 0
NaN values in rank: 0
NaN values in date: 0
NaN values in artist: 0
NaN values in url: 0
NaN values in region: 0
NaN values in chart: 0
NaN values in trend: 0
NaN values in streams: 0


In [22]:
# checking if there is a memory usage saving
dataframe.info(memory_usage='deep')




<class 'pandas.core.frame.DataFrame'>
Int64Index: 26147923 entries, 0 to 498949
Data columns (total 9 columns):
 #   Column   Dtype         
---  ------   -----         
 0   title    category      
 1   rank     int32         
 2   date     datetime64[ns]
 3   artist   category      
 4   url      category      
 5   region   category      
 6   chart    category      
 7   trend    category      
 8   streams  int32         
dtypes: category(6), datetime64[ns](1), int32(2)
memory usage: 1.0 GB


## 1.3 Charts plotting 

In [23]:
# data filters 

# sample data
choosen_start_date = '2019-05-05'
choosen_end_date = '2021-12-05'
choosen_artist = 'Drake'
choosen_title = 'bad guy'
choosen_region = 'United States'
choosen_trend = 'MOVE_UP'
choosen_top_rank = 20
choosen_streams_more_than = 1000
choosen_chart = 'top200'

# sample filters
data_filter = dataframe['date'].between(choosen_start_date, choosen_end_date)
artist_filter = dataframe['artist'] == choosen_artist
title_filter = dataframe['title'] == choosen_title
region_filter = dataframe['region'] == choosen_region
trend_filter = dataframe['trend'] == choosen_trend
top_rank_filter = dataframe['rank'] <= choosen_top_rank
more_than_streams_filter = dataframe['streams'] >= choosen_streams_more_than
chart_filter = dataframe['chart'] == choosen_chart

In [27]:
# rank changes over time for given parameters
import plotly.graph_objects as go

choosen_start_date = '2019-02-05'
choosen_end_date = '2019-09-05'
choosen_artist = "Billy Eilish"
choosen_title = "bad guy"
choosen_region = 'United States'
choosen_chart = 'top200'

data_filter = dataframe['date'].between(choosen_start_date, choosen_end_date)
artist_filter = dataframe['artist'] == choosen_artist
title_filter = dataframe['title'] == choosen_title
region_filter = dataframe['region'] == choosen_region
chart_filter = dataframe['chart'] == choosen_chart

# filtered dataframe
rank_changes_data_dataframe = dataframe[data_filter & region_filter & chart_filter & title_filter].sort_values(by='date')

# chart plotting
full_title_fig_1 = f'{choosen_artist} - {choosen_title} ranking in {choosen_chart} chart in \
{choosen_region} from {choosen_start_date} to {choosen_end_date}.'
print(full_title_fig_1)


fig_1 = px.line(template='plotly_dark')
fig_1.add_trace(go.Scatter(x=rank_changes_data_dataframe['date'], y=rank_changes_data_dataframe['rank'], 
                           name=f'{choosen_artist} - {choosen_title}', line=dict(color="#1DB954"), showlegend=True))
fig_1.update_layout(title="Ranking", 
    title_x=0.5,
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig_1.show()

Billy Eilish - bad guy ranking in top200 chart in United States from 2019-02-05 to 2019-09-05.


In [30]:
# rank changes over time for given parameters (song comparison)
# item 1
choosen_start_date = '2019-02-05'
choosen_end_date = '2019-09-05'
choosen_artist = "Billy Eilish"
choosen_title = "bad guy"
choosen_region = 'United States'
choosen_chart = 'top200'

data_filter = dataframe['date'].between(choosen_start_date, choosen_end_date)
artist_filter = dataframe['artist'] == choosen_artist
title_filter = dataframe['title'] == choosen_title
region_filter = dataframe['region'] == choosen_region
chart_filter = dataframe['chart'] == choosen_chart

# item 2
choosen_start_date_2 = '2019-05-05'
choosen_end_date_2 = '2019-09-05'
choosen_artist_2 = "Drake"
choosen_title_2 = "God's Plan"
choosen_region_2 = 'United States'
choosen_chart_2 = 'top200'

data_filter_2 = dataframe['date'].between(choosen_start_date_2, choosen_end_date_2)
artist_filter_2 = dataframe['artist'] == choosen_artist_2
title_filter_2 = dataframe['title'] == choosen_title_2
region_filter_2 = dataframe['region'] == choosen_region_2
chart_filter_2 = dataframe['chart'] == choosen_chart_2

# filtered dataframe
# dataframe 1 
rank_changes_data_dataframe = dataframe[data_filter & region_filter & chart_filter & title_filter].sort_values(by='date')
# dataframe 2
rank_changes_data_dataframe_2 = dataframe[data_filter_2 & region_filter_2 & chart_filter_2 & title_filter_2].sort_values(by='date')

# chart plotting
full_title_fig_2 =f"{choosen_artist} - {choosen_title} ranking in {choosen_chart} chart in \
{choosen_region} from {choosen_start_date} to {choosen_end_date} and \
{choosen_artist_2} - {choosen_title_2} ranking in {choosen_chart_2} chart in \
{choosen_region_2} from {choosen_start_date_2} to {choosen_end_date_2} comparison."
print(full_title_fig_2)

fig_2 = px.line(template='plotly_dark')
fig_2.add_trace(go.Scatter(x=rank_changes_data_dataframe['date'], y=rank_changes_data_dataframe['rank'], 
                           name=f'{choosen_artist} - {choosen_title}', line=dict(color="#1DB954")))
fig_2.add_trace(go.Scatter(x=rank_changes_data_dataframe_2['date'], y=rank_changes_data_dataframe['rank'], 
                           name=f'{choosen_artist_2} - {choosen_title_2}', line=dict(color="#1771F1")))
fig_2.update_layout(
    title="Song titles ranking comparison",
    title_x=0.5,
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1))
fig_2.show()


Billy Eilish - bad guy ranking in top200 chart in United States from 2019-02-05 to 2019-09-05 and Drake - God's Plan ranking in top200 chart in United States from 2019-05-05 to 2019-09-05 comparison.


In [41]:
# how many times song came up in rank for given parameters 

choosen_start_date = '2018-01-05'
choosen_end_date = '2019-12-05'
choosen_artist = 'Drake'
choosen_region = 'United States'
choosen_chart = 'top200'
choosen_top_rank = 20

data_filter = dataframe['date'].between(choosen_start_date, choosen_end_date)
artist_filter = dataframe['artist'] == choosen_artist
top_rank_filter = dataframe['rank'] <= choosen_top_rank
region_filter = dataframe['region'] == choosen_region
chart_filter = dataframe['chart'] == choosen_chart

# filtered dataframe
titles_rank_changes_dataframe = dataframe[data_filter & region_filter & chart_filter & artist_filter & top_rank_filter]

# calculation of unique titles in filtered dataframe
number_of_unique_songs = len(titles_rank_changes_dataframe['title'].unique())
print(f'Number of unique songs for given parameters: {number_of_unique_songs}')

# preparing new dataframe containing required data
number_of_title_occurance_series = pd.Series(titles_rank_changes_dataframe['title'].value_counts())
number_of_title_occurance_dataframe = number_of_title_occurance_series.to_frame().reset_index()
number_of_title_occurance_dataframe.rename(columns={'index':'title', 'title':'number_of_title_occurance'},
          inplace = True)
number_of_title_occurance_dataframe_no_zeros_filter = number_of_title_occurance_dataframe['number_of_title_occurance'] > 0
number_of_title_occurance_dataframe = number_of_title_occurance_dataframe[number_of_title_occurance_dataframe_no_zeros_filter]

# chart plotting
full_title_fig_3 = f"{choosen_artist}'s songs occurance in the first {choosen_top_rank} of \
{choosen_chart} chart in {choosen_region} from {choosen_start_date} to {choosen_end_date}."
print(full_title_fig_3)


fig_3 = px.bar(template='plotly_dark')
fig_3.add_trace(go.Bar(x=number_of_title_occurance_dataframe['title'], y=number_of_title_occurance_dataframe['number_of_title_occurance'], 
                           name=f'{choosen_artist}', marker_color='#1DB954', showlegend=True))
fig_3.update_layout(title="Songs occurance in the ranking",
    title_x=0.5,
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1))
fig_3.show()



Number of unique songs for given parameters: 26
Drake's songs occurance in the first 20 of top200 chart in United States from 2018-01-05 to 2019-12-05.


In [57]:
# how many times song came up in rank for given parameters (artist comparison)

# item 1
choosen_start_date = '2019-01-01'
choosen_end_date = '2020-01-01'
choosen_artist = 'Drake'
choosen_region = 'United States'
choosen_chart = 'top200'
choosen_top_rank = 20

data_filter = dataframe['date'].between(choosen_start_date, choosen_end_date)
artist_filter = dataframe['artist'] == choosen_artist
top_rank_filter = dataframe['rank'] <= choosen_top_rank
region_filter = dataframe['region'] == choosen_region
chart_filter = dataframe['chart'] == choosen_chart

# item 2
choosen_start_date_2 = choosen_start_date
choosen_end_date_2 = choosen_end_date
choosen_artist_2 = 'Kanye West'
choosen_region_2 = 'United States'
choosen_chart_2 = 'top200'
choosen_top_rank_2 = choosen_top_rank

data_filter_2 = dataframe['date'].between(choosen_start_date_2, choosen_end_date_2)
artist_filter_2 = dataframe['artist'] == choosen_artist_2
top_rank_filter_2 = dataframe['rank'] <= choosen_top_rank_2
region_filter_2 = dataframe['region'] == choosen_region_2
chart_filter_2 = dataframe['chart'] == choosen_chart_2

# filtered dataframe
# dataframe 1
titles_rank_changes_dataframe = dataframe[data_filter & region_filter & chart_filter & artist_filter & top_rank_filter]
# dataframe 2
titles_rank_changes_dataframe_2 = dataframe[data_filter_2 & region_filter_2 & chart_filter_2 & artist_filter_2 & top_rank_filter_2]

# calculation of unique titles in filtered dataframe
# dataframe 1
number_of_unique_songs = len(titles_rank_changes_dataframe['title'].unique())
print(f'Number of unique songs for given parameters ({choosen_artist}): {number_of_unique_songs}')
# dataframe 2
number_of_unique_songs_2 = len(titles_rank_changes_dataframe_2['title'].unique())
print(f'Number of unique songs for given parameters ({choosen_artist_2}): {number_of_unique_songs_2}')

# preparing new dataframe containing required data
# new dataframe item 1
number_of_title_occurance_series = pd.Series(titles_rank_changes_dataframe['title'].value_counts())
number_of_title_occurance_dataframe = number_of_title_occurance_series.to_frame().reset_index()
number_of_title_occurance_dataframe.rename(columns={'index':'title', 'title':'number_of_title_occurance'},
          inplace = True)
number_of_title_occurance_dataframe_no_zeros_filter = number_of_title_occurance_dataframe['number_of_title_occurance'] > 0
number_of_title_occurance_dataframe = number_of_title_occurance_dataframe[number_of_title_occurance_dataframe_no_zeros_filter]
# new dataframe item 2
number_of_title_occurance_series_2 = pd.Series(titles_rank_changes_dataframe_2['title'].value_counts())
number_of_title_occurance_dataframe_2 = number_of_title_occurance_series_2.to_frame().reset_index()
number_of_title_occurance_dataframe_2.rename(columns={'index':'title', 'title':'number_of_title_occurance'},
          inplace = True)
number_of_title_occurance_dataframe_no_zeros_filter_2 = number_of_title_occurance_dataframe_2['number_of_title_occurance'] > 0
number_of_title_occurance_dataframe_2 = number_of_title_occurance_dataframe_2[number_of_title_occurance_dataframe_no_zeros_filter_2]

# chart plotting
full_title_fig_4 = f"{choosen_artist}'s songs occurance in the first {choosen_top_rank} of \
{choosen_chart} chart in {choosen_region} from {choosen_start_date} to {choosen_end_date} and \
{choosen_artist_2}'s songs occurance in the first {choosen_top_rank_2} of \
{choosen_chart_2} chart in {choosen_region_2} from {choosen_start_date_2} to {choosen_end_date_2} comparison"
print(full_title_fig_4)


fig_4 = px.bar(template='plotly_dark')
fig_4.add_trace(go.Bar(x=number_of_title_occurance_dataframe['title'], y=number_of_title_occurance_dataframe['number_of_title_occurance'], 
                           name=f'{choosen_artist}', marker_color='#1DB954', showlegend=True))
fig_4.add_trace(go.Bar(x=number_of_title_occurance_dataframe_2['title'], y=number_of_title_occurance_dataframe_2['number_of_title_occurance'], 
                           name=f'{choosen_artist_2}', marker_color='#1771F1', showlegend=True))
fig_4.update_layout(title="Songs occurance in the ranking - artists comparison",
    title_x=0.5,
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1))
fig_4.show()

Number of unique songs for given parameters (Drake): 6
Number of unique songs for given parameters (Kanye West): 6
Drake's songs occurance in the first 20 of top200 chart in United States from 2019-01-01 to 2020-01-01 and Kanye West's songs occurance in the first 20 of top200 chart in United States from 2019-01-01 to 2020-01-01 comparison
